Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace here.

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. You can read more about the interest rate that Lending Club assigns here. Lending Club also tries to verify each piece of information the borrower provides but it can't always verify all of the information (usually for regulation reasons).

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiveing a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.

While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. While at first, you may wonder why investors would put money into anything but low interest loans. The incentive investors have to back higher interest loans is, well, the higher interest! If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this project, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.


Data dictionary: https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097

*** Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not? ***

First Let's load the data and do some quick checks:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
loan_df = pd.read_csv(".\\data\\LoanStats3a.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan_df.shape

(42538, 111)

In [4]:
loan_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


Let's start getting rid of useless columns, by useless columns I mean columns that:

* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information

removing the desc column (which contains a long text explanation for each loan) and removing the url column (which contains a link to each loan on Lending Club which can only be accessed with an investor account)

In [5]:
loan_df = loan_df.drop(['desc', 'url'],axis=1)

removing duplicates if any

In [6]:
print(loan_df.shape)
print("Removing duplicates if any ...")
loan_df.drop_duplicates()
print(loan_df.shape)

(42538, 109)
Removing duplicates if any ...
(42538, 109)


Dropping columns that are reduntant or leak data

In [7]:
loan_df.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade',
              'issue_d','zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv',
              'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee',
              'last_pymnt_d','last_pymnt_amnt'], axis=1,inplace=True)

In [8]:
loan_df.shape

(42538, 90)

Dropping columns with more than 90% missing values

In [9]:
# check for columns with 90% of missing values
percent = 0.9
over_ninety_percernt_ma = loan_df.isnull().sum()[loan_df.isnull().sum() /  loan_df.shape[0] > percent].index.tolist()
loan_df.drop(over_ninety_percernt_ma, axis=1,inplace=True)
over_ninety_percernt_ma.sort()
over_ninety_percernt_ma

['acc_open_past_24mths',
 'all_util',
 'annual_inc_joint',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'dti_joint',
 'il_util',
 'inq_fi',
 'inq_last_12m',
 'max_bal_bc',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_last_major_derog',
 'mths_since_last_record',
 'mths_since_rcnt_il',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'next_pymnt_d',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'open_acc_6m',
 'open_il_12m',
 'open_il_24m',
 'open_il_6m',
 'open_rv_12m',
 'open_rv_24m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'tot_coll_amt',
 'tot_cur_bal',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bal_il',
 'total_bc_l

In [10]:
loan_df.shape

(42538, 34)

In [11]:
loan_df.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

Now let's focus on the target column.

Since we're interesting in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case

In [12]:
#target coluumn
loan_df.loan_status.value_counts()

Fully Paid                                             33586
Charged Off                                             5653
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Current                                                  513
In Grace Period                                           16
Late (31-120 days)                                        12
Late (16-30 days)                                          5
Default                                                    1
Name: loan_status, dtype: int64

In [13]:
# keep "Full Paid" and "Charged Off"
loan_df = loan_df[loan_df["loan_status"].isin(["Fully Paid","Charged Off"])]
mapping_dict = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}
loan_df = loan_df.replace(mapping_dict)
loan_df.loan_status.value_counts()

1    33586
0     5653
Name: loan_status, dtype: int64

let's also get rid of columns that only has 1 unique value (that won't give our model any info)

In [14]:
# let's check for columns with just one value
cols_to_drop = []
for col in loan_df.columns:
    if len(loan_df[col].dropna().unique()) == 1:
           cols_to_drop.append(col)

print(cols_to_drop)
loan_df.drop(cols_to_drop, axis=1, inplace=True)    

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [15]:
# this one has only 1 value for 1 ....
print(loan_df.pymnt_plan.value_counts())
loan_df.drop('pymnt_plan', axis=1, inplace=True) # get rid of it

n    39238
y        1
Name: pymnt_plan, dtype: int64


Now we are going to work on the features that still have missing values, we can:

* remove rows
* set new value (other, min, max, mean, etc ..)
* regress new value base on other columns
* ...


In [16]:
loan_df.isnull().sum()

loan_amnt                     0
term                          0
int_rate                      0
installment                   0
emp_title                  2426
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
loan_status                   0
purpose                       0
title                        10
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
mths_since_last_delinq    25352
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                   50
total_acc                     0
last_credit_pull_d            2
pub_rec_bankruptcies        697
dtype: int64

In [17]:
loan_df.title.value_counts().head() # 10 remove!

Debt Consolidation         2144
Debt Consolidation Loan    1671
Personal Loan               650
Consolidation               502
debt consolidation          495
Name: title, dtype: int64

In [18]:
loan_df.pub_rec_bankruptcies.value_counts()  # substititue for 0 697        

0.0    36872
1.0     1665
2.0        5
Name: pub_rec_bankruptcies, dtype: int64

In [19]:
loan_df.revol_util.value_counts().head() # delete missing values 50

0%       972
0.2%      63
63%       62
40.7%     59
0.1%      58
Name: revol_util, dtype: int64

In [20]:
print(loan_df.mths_since_last_delinq.median())
print(loan_df.mths_since_last_delinq.mean())
# 25352 substitute with mean!!!!! maybe try something different afterwards

34.0
35.8845683013


In [21]:
loan_df.last_credit_pull_d.value_counts().head() # delete missing values 2       

Sep-2016    14555
Mar-2016      827
Aug-2016      716
Apr-2016      664
Feb-2013      663
Name: last_credit_pull_d, dtype: int64

In [22]:
loan_df.emp_title.value_counts().head() #remove it for now 

US Army              134
Bank of America      109
IBM                   66
AT&T                  58
Kaiser Permanente     57
Name: emp_title, dtype: int64

After checking the variables let's manage missing values

In [23]:
# fix missing values in columns
loan_df.drop('emp_title', axis=1, inplace=True)
loan_df.mths_since_last_delinq = loan_df.mths_since_last_delinq.fillna(loan_df.mths_since_last_delinq.median())
loan_df.pub_rec_bankruptcies = loan_df.pub_rec_bankruptcies.fillna(0)
# remove rows with missing values
loan_df.dropna(axis=0, inplace=True)
# check missing values again (should be none)
loan_df.isnull().sum()

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                     0
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
mths_since_last_delinq    0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util                0
total_acc                 0
last_credit_pull_d        0
pub_rec_bankruptcies      0
dtype: int64

Let's focus now on the text columns. We need to convert them to numerical 

In [24]:
print(loan_df.dtypes.value_counts())
# show an example of object columns
loan_df.select_dtypes(include=['object']).iloc[0]

float64    12
object     11
int64       1
dtype: int64


term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Sep-2016
Name: 0, dtype: object

Let's first convert revol_util and int_rate to numerical values

In [25]:
loan_df.int_rate = loan_df.int_rate.map(lambda x: float(x[:-1]))
loan_df.revol_util = loan_df.revol_util.map(lambda x: float(x[:-1]))
loan_df.term = loan_df.term.map(lambda x: int(x.split()[0]))

In [63]:
loan_df.select_dtypes(include=['object']).iloc[0]

Series([], Name: 0, dtype: float64)

In [27]:
# term
loan_df.emp_length.value_counts()
mapping_dict = {'emp_length':{
        '10+ years':'10',
        '< 1 year':'0',
        'n/a':'-1'
    }
}

loan_df = loan_df.replace(mapping_dict)
loan_df.emp_length = loan_df.emp_length.map(lambda x: int(x.split()[0]))
loan_df.emp_length.value_counts()

 10    8715
 0     4542
 2     4344
 3     4050
 4     3385
 5     3243
 1     3207
 6     2198
 7     1738
 8     1457
 9     1245
-1     1053
Name: emp_length, dtype: int64

In [29]:
# home_ownership 
loan_df.home_ownership.value_counts()

RENT        18677
MORTGAGE    17381
OWN          3020
OTHER          96
NONE            3
Name: home_ownership, dtype: int64

In [30]:
loan_df.verification_status.value_counts()

Not Verified       16809
Verified           12515
Source Verified     9853
Name: verification_status, dtype: int64

In [36]:
loan_df.purpose.value_counts()

debt_consolidation    18355
credit_card            5073
other                  3921
home_improvement       2944
major_purchase         2178
small_business         1792
car                    1534
wedding                 940
medical                 688
moving                  580
vacation                377
house                   372
educational             320
renewable_energy        103
Name: purpose, dtype: int64

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

In [34]:
loan_df.drop('title',axis=1, inplace=True)

purpose, verification_status, home_ownership, and addr_state as dummies because they don't have many values (maybe we could skip state)

In [42]:
loan_df = pd.concat([loan_df, pd.get_dummies(loan_df.purpose)], axis=1)
loan_df = pd.concat([loan_df, pd.get_dummies(loan_df.verification_status)], axis=1)
loan_df = pd.concat([loan_df, pd.get_dummies(loan_df.home_ownership)], axis=1)
loan_df = pd.concat([loan_df, pd.get_dummies(loan_df.addr_state)], axis=1)
loan_df.drop(['purpose','verification_status','home_ownership','addr_state'], axis=1, inplace=True)

for earliest_cr_line, last_credit_pull_d we want to get month and year in numbers, I can do and float that will tell me year and month

In [56]:
# convert to date
earliest_cr_line = pd.to_datetime(loan_df.earliest_cr_line)
last_credit_pull_d = pd.to_datetime(loan_df.last_credit_pull_d)
# extract month and year
earliest_cr_line_year = earliest_cr_line.map(lambda x: x.year)
earliest_cr_line_month = earliest_cr_line.map(lambda x: x.month)
last_credit_pull_d_year = last_credit_pull_d.map(lambda x: x.year)
last_credit_pull_d_month = last_credit_pull_d.map(lambda x: x.month)
# create float with year + month/13
loan_df.earliest_cr_line = earliest_cr_line_year + earliest_cr_line_month/13
loan_df.last_credit_pull_d = last_credit_pull_d_year + last_credit_pull_d_month/13

In [64]:
# save processed data
loan_df.to_csv(".\\data_processed\loans.csv")

In [ ]:
# https://www.dataquest.io/mission/213/machine-learning-project-walkthrough-data-cleaning